In [10]:
from IPython.core.display import display, HTML

display(HTML("<style>.container { width:80% !important; }</style>"))

%load_ext autoreload
%autoreload 2
%load_ext blackcellmagic
%config Completer.use_jedi = False

import pyphi
from visualize_pyphi import *
from visualize_pyphi import utils
from visualize_pyphi.compute import *
import pickle as pkl
from tqdm.auto import tqdm
import numpy as np
import itertools
from pyphi.models.subsystem import FlatCauseEffectStructure as sep
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))
import pandas as pd
pd.set_option('display.max_rows', None)

'''
Check that the config parameters are correct (if not change pyphi_config.yml to match):
REPERTOIRE_DISTANCE = 'maximal-state-first'
IIT_VERSION = 'IIT_4.0_SMALL_PHI'
PARTITION_TYPE = 'TRI'
'''
pyphi.config.REPERTOIRE_DISTANCE,pyphi.config.IIT_VERSION,pyphi.config.PARTITION_TYPE


C = pyphi.Direction.CAUSE
E = pyphi.Direction.EFFECT


/tmp/ipykernel_143107/2425759611.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The blackcellmagic extension is already loaded. To reload it, use:
  %reload_ext blackcellmagic


/tmp/ipykernel_143107/2425759611.py:19: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


# Space

In [63]:
with open(r"example_data/4_node_grid_subsystem.pkl", "rb") as f:
    subsystem = pkl.load(f)

In [65]:
ces = pyphi.compute.ces(subsystem)

In [66]:
relations = list(pyphi.relations.relations(subsystem,sep(ces),max_degree=2))

  0%|          | 0/334 [00:00<?, ?it/s]

In [67]:
sia = pyphi.big_phi.sia(subsystem,ces,relations)

Done putting relations
Finding maximal compositional state


Submitting tasks: 0it [00:00, ?it/s]

Done submitting tasks


  0%|          | 0/2 [00:00<?, ?it/s]

Evaluating maximal compositional state


In [68]:
sia.phi

0.014032332339088526

In [69]:
mechanisms = list(pyphi.utils.powerset(subsystem.node_indices,nonempty=True))
purviews = list(pyphi.utils.powerset(subsystem.node_indices,nonempty=True))
bag_of_mices = get_bag_of_mices(subsystem,mechanisms,purviews,candidate='miw')

  0%|          | 0/15 [00:00<?, ?it/s]

In [70]:
flat_bag = []
for mechanism in tqdm(mechanisms):
    for direction in [C,E]:
        for purview in mechanisms: 
            try:
                flat_bag.append(bag_of_mices[mechanism][direction][purview])
            except:
                pass

reflexive_ds = [
    pyphi.subsystem.Concept(m1.mechanism, m1, m2, subsystem)
    for m1,m2 in itertools.combinations(flat_bag,2)
    if m1.mechanism==m2.mechanism==m1.purview==m2.purview
    and m1.direction == C
    and m2.direction == E
]

reflexive_sep = sep(pyphi.models.CauseEffectStructure(reflexive_ds))

reflexive_causes = [m.purview for m in reflexive_sep if m.direction==C]
reflexive_effects = [m.purview for m in reflexive_sep if m.direction==E]
reflexive_mechanisms = [m.mechanism for m in reflexive_sep]

remaining_mices = []
for m in flat_bag:
    if m.direction==C and m.mechanism not in reflexive_mechanisms and m.purview not in reflexive_causes:
        remaining_mices.append(m)
    if m.direction==E and m.mechanism not in reflexive_mechanisms and m.purview not in reflexive_effects:
        remaining_mices.append(m)

remaining_ces = CES_from_bag_of_mices(make_bag(remaining_mices))

final_ces = compute.get_linked_ces(list(reflexive_sep)+list(remaining_ces),subsystem)
final_ces.subsystem=subsystem

  0%|          | 0/15 [00:00<?, ?it/s]

In [71]:
relations = list(pyphi.relations.relations(subsystem,sep(final_ces),max_degree=2,parallel=True))

  0%|          | 0/237 [00:00<?, ?it/s]

/home/mgrasso/pyphi/pyphi/utils.py:86: RuntimeWarning: invalid value encountered in double_scalars
  return abs(x - y) <= constants.EPSILON
/home/mgrasso/pyphi/pyphi/utils.py:86: RuntimeWarning: invalid value encountered in double_scalars
  return abs(x - y) <= constants.EPSILON
/home/mgrasso/pyphi/pyphi/utils.py:86: RuntimeWarning: invalid value encountered in double_scalars
  return abs(x - y) <= constants.EPSILON
/home/mgrasso/pyphi/pyphi/utils.py:86: RuntimeWarning: invalid value encountered in double_scalars
  return abs(x - y) <= constants.EPSILON
/home/mgrasso/pyphi/pyphi/utils.py:86: RuntimeWarning: invalid value encountered in double_scalars
  return abs(x - y) <= constants.EPSILON
/home/mgrasso/pyphi/pyphi/utils.py:86: RuntimeWarning: invalid value encountered in double_scalars
  return abs(x - y) <= constants.EPSILON
/home/mgrasso/pyphi/pyphi/utils.py:86: RuntimeWarning: invalid value encountered in double_scalars
  return abs(x - y) <= constants.EPSILON
/home/mgrasso/pyphi/

In [72]:
sia = pyphi.big_phi.sia(subsystem,final_ces,relations)

sia.phi

Done putting relations
Finding maximal compositional state


Submitting tasks: 0it [00:00, ?it/s]

Done submitting tasks


  0%|          | 0/1 [00:00<?, ?it/s]

Evaluating maximal compositional state


0.04542253372529282

In [73]:
utils.sepces2df(sep(sia.phi_structure.distinctions),subsystem)

,mechanism,direction,purview,state,phi
0,A,CAUSE,A,0,0.520382
1,A,EFFECT,A,0,0.656129
2,B,CAUSE,B,0,0.629285
3,B,EFFECT,B,0,0.713724
4,C,CAUSE,C,0,0.629285
5,C,EFFECT,C,0,0.713724
6,D,CAUSE,D,0,0.520382
7,D,EFFECT,D,0,0.656129
8,AB,CAUSE,AB,00,0.071485
9,AB,EFFECT,AB,00,0.069119


# Ch 3

## 4 nodes

In [97]:
import pyphi
from visualize_pyphi import *
import numpy as np
from pyphi.models.subsystem import FlatCauseEffectStructure as sep

th = 1/4
exp = 5
mu = 1
si = .3
ll = 1
kk = 15
x0 = .7

q = .4
p = .1
s = .8
l = .25
j = 1-q
i = j-.05
b = .01

weights = np.array([
    [s,l,0,i,0,0],#A
    [l,s,l,j,b,0],#B
    [0,l,s,i,0,0],#C
    [b,b,b,q,0,i],#D
    [0,i,0,0,0,0],#I
    [0,0,0,b,0,s],#O
    #A,B,C,D,I,O
  ])

node_labels = ['A','B','C','D','I','O']
mech_func = [f for f in 'sssgll']
network = network_generator.get_net(mech_func, weights, exp=exp, th=th, mu=mu, si=si, l=ll, k=kk, x0=x0, node_labels=node_labels, pickle_network=False)
# utils.pklthis(network,'example_data/ch3_network.pkl')

subsystem = pyphi.Subsystem(network,state=(0,1,0,1,1,1),nodes=(0,1,2,3))
ces = pyphi.compute.ces(subsystem)
# utils.sepces2df(sep(ces),subsystem)

mechanisms = list(pyphi.utils.powerset(subsystem.node_indices,nonempty=True))
purviews = list(pyphi.utils.powerset(subsystem.node_indices,nonempty=True))
bag_of_mices = get_bag_of_mices(subsystem,mechanisms,purviews,candidate='miw')

pruned_ces = compute.get_linked_ces(CES_from_bag_of_mices(bag_of_mices),subsystem)
utils.sepces2df(sep(pruned_ces),subsystem)

  0%|          | 0/64 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

,mechanism,direction,purview,state,phi
0,A,CAUSE,A,0,0.486034
1,A,EFFECT,A,0,0.627340
2,B,CAUSE,B,1,0.232803
3,B,EFFECT,B,1,0.352352
4,C,CAUSE,C,0,0.486034
5,C,EFFECT,C,0,0.627340
6,D,CAUSE,AC,10\n01,0.103475
7,D,EFFECT,AC,00,0.005182
8,AD,CAUSE,AB,01,0.140774
9,AD,EFFECT,D,1,0.130094


In [98]:
pruned_relations = list(pyphi.relations.relations(subsystem,pruned_ces,max_degree=3))

pruned_sia = pyphi.big_phi.sia(subsystem,pruned_ces,pruned_relations)
pruned_sia_ces = pruned_sia.phi_structure.distinctions
pruned_sia_rels = pruned_sia.phi_structure.relations
pruned_sia.phi

  0%|          | 0/355 [00:00<?, ?it/s]

/home/mgrasso/pyphi/pyphi/utils.py:86: RuntimeWarning:

invalid value encountered in double_scalars



Done putting relations
Finding maximal compositional state


Submitting tasks: 0it [00:00, ?it/s]

Done submitting tasks


  0%|          | 0/1 [00:00<?, ?it/s]

Evaluating maximal compositional state


0.00012485547369374585

In [99]:
fig = visualize_ces.plot_ces(
    subsystem,
    sep(pruned_sia_ces),
    pruned_sia_rels,
    base_rotation=0,
    base_z_offset=.3,
)
fig.show()

Computing edges:   0%|          | 0/51 [00:00<?, ?it/s]

Computing triangles:   0%|          | 0/64 [00:00<?, ?it/s]

Fig saved to .html


## 5 nodes

In [52]:
subsystem = pyphi.Subsystem(network,state=(0,1,0,1,1,1),nodes=(0,1,2,3,5))
ces = pyphi.compute.ces(subsystem)
utils.sepces2df(sep(ces),subsystem)

,mechanism,direction,purview,state,phi
0,A,CAUSE,A,0,0.615201
1,A,EFFECT,A,0,0.671364
2,B,CAUSE,BD,00,0.037179
3,B,EFFECT,BD,10,0.223010
4,C,CAUSE,C,0,0.615201
5,C,EFFECT,C,0,0.671364
6,D,CAUSE,ABC,100\n001,0.433502
7,D,EFFECT,BO,11,0.957110
8,O,CAUSE,D,1,0.999928
9,O,EFFECT,D,0,0.108492


In [53]:
relations = list(pyphi.relations.relations(subsystem,sep(ces),max_degree=2))
sia = pyphi.big_phi.sia(subsystem,ces,relations)
sia.phi

  0%|          | 0/1373 [00:00<?, ?it/s]

/home/mgrasso/pyphi/pyphi/utils.py:86: RuntimeWarning: invalid value encountered in double_scalars
  return abs(x - y) <= constants.EPSILON


Done putting relations
Finding maximal compositional state


Submitting tasks: 0it [00:00, ?it/s]

2022-01-31 16:51:28,440	ERROR worker.py:84 -- Unhandled error (suppress with RAY_IGNORE_UNHANDLED_ERRORS=1): ray::_max_system_intrinsic_information() (pid=2222743, ip=144.92.2.85)
  File "/home/mgrasso/pyphi/pyphi/big_phi.py", line 600, in _max_system_intrinsic_information
    return max(
TypeError: '>' not supported between instances of 'PhiStructure' and 'PhiStructure'
2022-01-31 16:51:29,442	ERROR worker.py:84 -- Unhandled error (suppress with RAY_IGNORE_UNHANDLED_ERRORS=1): ray::_max_system_intrinsic_information() (pid=2222298, ip=144.92.2.85)
  File "/home/mgrasso/pyphi/pyphi/big_phi.py", line 600, in _max_system_intrinsic_information
    return max(
TypeError: '>' not supported between instances of 'PhiStructure' and 'PhiStructure'
2022-01-31 16:51:30,446	ERROR worker.py:84 -- Unhandled error (suppress with RAY_IGNORE_UNHANDLED_ERRORS=1): ray::_max_system_intrinsic_information() (pid=2222722, ip=144.92.2.85)
  File "/home/mgrasso/pyphi/pyphi/big_phi.py", line 600, in _max_system_

In [ ]:
utils.sepces2df(sep(sia.phi_structure.distinctions),subsystem)